# Determing the movement of people in a retail shopping center

IoT Challenge 2017, team TyR data generator was created to model generate and analyse data which could be observed  using the developed LoPy software.

The following code generates a csv file with the same formating as the one created using the designed LoPy software and randomly generates movement and consumer behavior. This data is then parsed and graphs are generated to assist in locating patterns of consumer behavior.

The following libraries will assist in data analysis:

In [62]:
import csv
import random
import pandas as pd
import numpy as np
import plotly.plotly as py
py.sign_in('jamesguyer', 'Y712MgscGHtjpZEcOzb3')
import plotly.graph_objs as go
from datetime import datetime, timedelta

%matplotlib inline

The loPy contains a timing method that counts seconrds from January 1st 2000 as epoch. We will need a function to extract these seconds into a date.

In [51]:
def extract_date(input_seconds):
    # LoPy start of your epoch
    base = datetime(2000, 1, 1)

    # Number of current seconds
    time = base + timedelta(seconds=input_seconds)
    
    # return the date and time
    return str(time.year) + '-' + str(time.month) + '-' + str(time.day) + ' ' + str(time.hour) + ':' + str(time.minute) + ':' + str(time.second)

Write a csv file containing purely random data.

In [56]:
class people_gen:
    def __init__(self,direction_swap):
        self.direction_swap = direction_swap
        self.swapcount = 0
        
    def timing_people(self):
        # generate a random time for people
        ordinary_time = random.uniform(0.5, 2)
        if ordinary_time > 1.8:
            return ("{0:.2f}".format(random.uniform(1, 15)))
        else:
            return ("{0:.2f}".format(ordinary_time))

    def direction_people(self):
        # determine direction of travel
        direction = random.randrange(11)
        if self.swapcount < self.direction_swap:
            if direction < 7:
                self.swapcount = self.swapcount + 1
                return 1
            else:
                return 0
        else:
            if direction < 7:
                self.swapcount = self.swapcount + 1
                return 0
            else:
                return 1
    
class timeseconds:
    def __init__(self,start):
        # testing start time in seconds
        self.seconds = start

    def next_time(self):
        # generate a following time
        self.seconds = self.seconds + random.uniform(1, 15)
        return extract_date(self.seconds)

    
def test_data_write(max_lines):
    # time now in seconds since Jan 1st 2000
    start_time = 541183209
    # time control object
    ts = timeseconds(start_time)
    # after how many entries to change bias direction
    direction_swap = max_lines/2
    # people generator object
    pg = people_gen(direction_swap)
    # open csv file for outputting test data
    with open('testdata.csv', 'wb') as csvfile:
        testwriter = csv.writer(csvfile, delimiter=',')
        testwriter.writerow(['Timecode','Sensor_ID','Time_Stopped','Right_Pass','Left_Pass'])
        for x in range(0, max_lines):
            if pg.direction_people() == 1:
                testwriter.writerow([str(ts.next_time()),str(1),str(pg.timing_people()),str(0),str(1)])
            else:
                testwriter.writerow([str(ts.next_time()),str(1),str(pg.timing_people()),str(1),str(0)])
        print("Generated CSV Test File")
        
# Generate CSV with 2000 entries
test_data_write(2000)

Generated CSV Test File


Read in the CSV data for data operations

In [57]:
# Import data from csv
df = pd.read_csv('testdata.csv')
df.head()

,Timecode,Sensor_ID,Time_Stopped,Right_Pass,Left_Pass
0,2017-2-23 16:40:21,1,1.49,0,1
1,2017-2-23 16:40:23,1,1.71,1,0
2,2017-2-23 16:40:27,1,1.48,0,1
3,2017-2-23 16:40:39,1,1.61,0,1
4,2017-2-23 16:40:43,1,1.17,1,0


Plot the test data for the amount of time spent at location 1.

In [70]:
trace = go.Scatter(x=df['Timecode'],y=df['Time_Stopped'],name='seconds')

layout = go.Layout(
                  title='View Time People Passing Feburary 23rd 2017',
                  plot_bgcolor='rgb(230, 230,230)',
                  showlegend=True
                  )

fig = go.Figure(data=[trace], layout=layout)

py.iplot(fig, filename='people_timing')

We could also view the people traveling left or right or other combinations of data.